In [1]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

#主要輸入接收新聞標題本身，即一個整數序列（每個整數編碼一個詞）。
#這些整數在1 到10,000 之間（10,000 個詞的詞彙表），且序列長度為100 個詞
#宣告一個 NAME 去定義Input
main_input = Input(shape=(100,), dtype='int32', name='main_input1')


# Embedding 層將輸入序列編碼為一個稠密向量的序列，
# 每個向量維度為 512。
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# LSTM 層把向量序列轉換成單個向量，
# 它包含整個序列的上下文信息
lstm_out = LSTM(32)(x)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
#插入輔助損失，使得即使在模型主損失很高的情況下，LSTM 層和Embedding 層都能被平穩地訓練
news_output = Dense(1, activation='sigmoid', name='news_out')(lstm_out)# #Dense就是常用的全连接层 activation是逐元素计算的激活函数

In [3]:
#輔助輸入數據與LSTM 層的輸出連接起來，輸入到模型
import keras
news_input = Input(shape=(5,), name='news_in')
x = keras.layers.concatenate([lstm_out, news_input])


# 堆疊多個全連接網路層
x = Dense(65, activation='relu')(x)
x = Dense(65, activation='relu')(x)
#作業解答: 新增兩層
x = Dense(65, activation='relu')(x)
x = Dense(65, activation='relu')(x)

# 最後添加主要的邏輯回歸層
main_output = Dense(1, activation='sigmoid', name='main_output1')(x)

In [4]:
# 宣告 MODEL API, 分別採用自行定義的 Input/Output Layer
#model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])
model = Model(inputs=[main_input, news_input], outputs=[main_output, news_output])

In [5]:
#請修改 Name 中, 自定義的 Layer 名稱

#增加一層全連階層

#宣告 MODEL API, 分別採用自行定義的 Input/Output Layer

#model.summary 查看 Layers stack
#model.compile(optimizer='rmsprop',
#              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
 #             loss_weights={'main_output': 1., 'aux_output': 0.2})
model.compile(optimizer='rmsprop',

              loss={'main_output1': 'binary_crossentropy', 'news_out': 'binary_crossentropy'},

              loss_weights={'main_output1': 1., 'news_out': 0.2})
#compile() 函数将一个字符串编译为字节代码。
#optimizer 優化器
#RMSprop則是「在學習率上依據梯度的大小對學習率進行加強或是衰減」。
#loss 損失函
#main 主要
#binary_crossentropy交叉熵損失函式

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input1 (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 512)     5120000     main_input1[0][0]                
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           69760       embedding_1[0][0]                
__________________________________________________________________________________________________
news_in (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
concatenat